In [1]:
import os
from pprint import pprint
import sys

import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
from scipy import stats
from scipy.signal import find_peaks, hilbert
import pickle

project_root = '..'
sys.path.append(project_root)

from sleeprnn.common import viz, constants
from sleeprnn.helpers import reader, plotter, misc, performer
from sleeprnn.detection import metrics, det_utils
from figs_thesis import fig_utils
from baselines_scripts.butils import get_partitions
from sleeprnn.detection.feeder_dataset import FeederDataset
from sklearn.linear_model import LinearRegression, HuberRegressor
from sleeprnn.data import utils

RESULTS_PATH = os.path.join(project_root, 'results')
BASELINES_PATH = os.path.join(project_root, 'resources', 'comparison_data', 'baselines_2021')

%matplotlib inline
viz.notebook_full_width()

param_filtering_fn = fig_utils.get_filtered_signal_for_event
param_frequency_fn = fig_utils.get_frequency_by_fft
param_amplitude_fn = fig_utils.get_amplitude_event

# Comparación REDv2-Time y REDv2-CWT

Métricas en cada base de datos (5CV): F1-score y mIoU entre ambos REDv2 y entre ellos mismos.

In [ ]:
models = [constants.V2_TIME, constants.V2_CWT1D]
print_model_names = {
    constants.V2_TIME: 'REDv2-Time',
    constants.V2_CWT1D: 'REDv2-CWT'
}
print_dataset_names = {
    (constants.MASS_SS_NAME, 1): "MASS-SS2-E1SS",
    (constants.MASS_SS_NAME, 2): "MASS-SS2-E2SS",
    (constants.MASS_KC_NAME, 1): "MASS-SS2-KC",
    (constants.MODA_SS_NAME, 1): "MASS-MODA",
    (constants.INTA_SS_NAME, 1): "INTA-UCH",
}

eval_configs = [
    dict(dataset_name=constants.MASS_SS_NAME, expert=1, strategy='5cv', seeds=3),
    dict(dataset_name=constants.MASS_SS_NAME, expert=2, strategy='5cv', seeds=3),
    dict(dataset_name=constants.MASS_KC_NAME, expert=1, strategy='5cv', seeds=3),
    dict(dataset_name=constants.MODA_SS_NAME, expert=1, strategy='5cv', seeds=3),
    dict(dataset_name=constants.INTA_SS_NAME, expert=1, strategy='5cv', seeds=3),
]
metrics_list = []
metrics_raw_list = []
for config in eval_configs:
    print("\nLoading", config)
    dataset = reader.load_dataset(config["dataset_name"], verbose=False)
    _, _, test_ids_list = get_partitions(dataset, config["strategy"], config["seeds"])
    n_folds = len(test_ids_list)
    average_mode = constants.MICRO_AVERAGE if (config["dataset_name"] == constants.MODA_SS_NAME) else constants.MACRO_AVERAGE
    # Collect predictions
    pred_dict = {}
    for model_version in models:
        tmp_dict = fig_utils.get_red_predictions(model_version, config["strategy"], dataset, config["expert"], verbose=False)
        # Retrieve only predictions, same format as baselines
        pred_dict[model_version] = {s: {} for s in dataset.all_ids}
        for k in range(n_folds):
            fold_subjects = tmp_dict[k][constants.TEST_SUBSET].all_ids
            fold_predictions = tmp_dict[k][constants.TEST_SUBSET].get_stamps()
            for s, pred in zip(fold_subjects, fold_predictions):
                pred_dict[model_version][s][k] = pred
    # Generate typical dict
    pred_dict_original = {}
    for model_version in models:
        pred_dict_original[model_version] = {}
        for k in range(n_folds):
            pred_dict_original[model_version][k] = {s: pred_dict[model_version][s][k] for s in test_ids_list[k]}
    # Generate surrogate model
    # Random permutation of fold assignments of predictions
    pred_dict_permuted = {}
    for model_version in models:
        pred_dict_permuted[model_version] = {}
        for i_sub, subject_id in enumerate(dataset.all_ids):
            byfold_preds = pred_dict[model_version][subject_id]
            subject_folds = list(byfold_preds.keys())
            subject_preds = [byfold_preds[k] for k in subject_folds]
            subject_folds = np.random.RandomState(seed=i_sub).permutation(subject_folds)
            pred_dict_permuted[model_version][subject_id] = {k: pred for k, pred in zip(subject_folds, subject_preds)}
    pred_dict_permuted_original = {}
    for model_version in models:
        pred_dict_permuted_original[model_version] = {}
        for k in range(n_folds):
            pred_dict_permuted_original[model_version][k] = {s: pred_dict_permuted[model_version][s][k] for s in test_ids_list[k]}
    
    # Performance
    table = {'Comparison': [], 'F1-score': [], 'Recall': [], 'Precision': [], 'mIoU': [], 'Fold': []}
    # Measure performance of model with itself
    for model_version in models:
        for k in range(n_folds):
            subject_ids = test_ids_list[k]
            events_list = [pred_dict_original[model_version][k][subject_id] for subject_id in subject_ids]
            detections_list = [pred_dict_permuted_original[model_version][k][subject_id] for subject_id in subject_ids]
            performance = fig_utils.compute_fold_performance(events_list, detections_list, average_mode)
            table['Comparison'].append('%s_vs_%s' % (model_version, model_version))
            table['F1-score'].append(performance['F1-score'])
            table['Recall'].append(performance['Recall'])
            table['Precision'].append(performance['Precision'])
            table['mIoU'].append(performance['mIoU'])
            table['Fold'].append(k)
    # Measure performance of time vs cwt
    for k in range(n_folds):
        subject_ids = test_ids_list[k]
        events_list = [pred_dict_original[models[0]][k][subject_id] for subject_id in subject_ids]
        detections_list = [pred_dict_original[models[1]][k][subject_id] for subject_id in subject_ids]
        performance = fig_utils.compute_fold_performance(events_list, detections_list, average_mode)
        table['Comparison'].append('%s_vs_%s' % (models[0], models[1]))
        table['F1-score'].append(performance['F1-score'])
        table['Recall'].append(performance['Recall'])
        table['Precision'].append(performance['Precision'])
        table['mIoU'].append(performance['mIoU'])
        table['Fold'].append(k)
    
    table = pd.DataFrame.from_dict(table)
    metrics_raw_list.append(table)
    mean_table = table.groupby(by=["Comparison"]).mean()[["F1-score", "mIoU"]].add_suffix("_mean")
    std_table = table.groupby(by=["Comparison"]).std(ddof=0)[["F1-score", "mIoU"]].add_suffix("_std")
    subgroup_stats_table = mean_table.join(std_table)
    subgroup_stats_table = subgroup_stats_table.reindex(sorted(subgroup_stats_table.columns), axis=1)
    subgroup_stats_table = subgroup_stats_table.reset_index()
    metrics_list.append(subgroup_stats_table)  
print("Done.")

In [ ]:
# Tabla Latex
comparisons_order = ["v2_time_vs_v2_time", "v2_cwt1d_vs_v2_cwt1d", "v2_time_vs_v2_cwt1d"]
comparisons_print_name = {
    "v2_time_vs_v2_time": "REDv2-Time vs REDv2-Time", 
    "v2_cwt1d_vs_v2_cwt1d": "REDv2-CWT vs REDv2-CWT", 
    "v2_time_vs_v2_cwt1d": "REDv2-Time vs REDv2-CWT"
}

print("Datos & Comparación & F1-score (%) & mIoU (%) \\\\")
for i_config, config in enumerate(eval_configs):
    print("")
    dataset_str = print_dataset_names[(config["dataset_name"], config["expert"])]
    metrics_dict = metrics_list[i_config].set_index('Comparison').to_dict('index')
    for comparison in comparisons_order:
        mets = metrics_dict[comparison]
        metric_str = "$%1.1f\pm %1.1f$ & $%1.1f\pm %1.1f$" % (
            100 * mets['F1-score_mean'], 100 * mets['F1-score_std'],
            100 * mets['mIoU_mean'], 100 * mets['mIoU_std'])
        row_str = "%s & %s & %s \\\\" % (dataset_str, comparisons_print_name[comparison], metric_str)
        print(row_str)
        
    # Statistical tests
    reference_comparison = comparisons_order[-1]
    print("P-value test against %s" % reference_comparison)
    table = metrics_raw_list[i_config]
    for comparison in comparisons_order[:-1]:
        print("%s:" % comparison.ljust(30), end='')
        for metric_name in ["F1-score", "mIoU"]:
            model_metrics = table[table["Comparison"] == comparison][metric_name].values
            reference_metrics = table[table["Comparison"] == reference_comparison][metric_name].values
            pvalue = stats.ttest_ind(model_metrics, reference_metrics, equal_var=False)[1]
            print(" P(%s) %1.4f" % (metric_name, pvalue), end='')
        print("")

# Ensamble de RED

Desempeño al ensamblar las detecciones de ambos modelos (y ensambles de un modelo consigo mismo haciendo el truco de la permutacion) con un AND, con un OR, o al promediar las probabilidades ajustadas antes de aplicar el umbral. Esto para ver si las pequeñas diferencias que tienen ayudan o no.

In [ ]:
models = [constants.V2_TIME, constants.V2_CWT1D]
print_model_names = {
    constants.V2_TIME: 'REDv2-Time',
    constants.V2_CWT1D: 'REDv2-CWT'
}
print_dataset_names = {
    (constants.MASS_SS_NAME, 1): "MASS-SS2-E1SS",
    (constants.MASS_SS_NAME, 2): "MASS-SS2-E2SS",
    (constants.MASS_KC_NAME, 1): "MASS-SS2-KC",
    (constants.MODA_SS_NAME, 1): "MASS-MODA",
    (constants.INTA_SS_NAME, 1): "INTA-UCH",
}

eval_configs = [
    dict(dataset_name=constants.MASS_SS_NAME, expert=1, strategy='5cv', seeds=3),
    dict(dataset_name=constants.MASS_SS_NAME, expert=2, strategy='5cv', seeds=3),
    dict(dataset_name=constants.MASS_KC_NAME, expert=1, strategy='5cv', seeds=3),
    dict(dataset_name=constants.MODA_SS_NAME, expert=1, strategy='5cv', seeds=3),
    dict(dataset_name=constants.INTA_SS_NAME, expert=1, strategy='5cv', seeds=3),
]
ensemble_criterion_list = [
    'and', 
    # 'or', 
    'avg'
]

metrics_list = []
metrics_raw_list = []
for config in eval_configs:
    print("\nLoading", config)
    dataset = reader.load_dataset(config["dataset_name"], verbose=False)
    _, _, test_ids_list = get_partitions(dataset, config["strategy"], config["seeds"])
    n_folds = len(test_ids_list)
    average_mode = constants.MICRO_AVERAGE if (config["dataset_name"] == constants.MODA_SS_NAME) else constants.MACRO_AVERAGE
    # Collect predictions
    pred_dict = {}
    for model_version in models:
        tmp_dict = fig_utils.get_red_predictions(
            model_version, config["strategy"], dataset, config["expert"], verbose=False)
        # Retrieve probas and stamps
        pred_dict[model_version] = {s: {} for s in dataset.all_ids}
        for k in range(n_folds):
            fold_subjects = tmp_dict[k][constants.TEST_SUBSET].all_ids
            fold_probas = tmp_dict[k][constants.TEST_SUBSET].get_probabilities(return_adjusted=True)
            fold_stamps = tmp_dict[k][constants.TEST_SUBSET].get_stamps()
            for s, proba, stamp in zip(fold_subjects, fold_probas, fold_stamps):
                pred_dict[model_version][s][k] = {'probability': proba, 'stamp': stamp} 
    # Generate typical dict
    pred_dict_original = {}
    for model_version in models:
        pred_dict_original[model_version] = {}
        for k in range(n_folds):
            pred_dict_original[model_version][k] = {s: pred_dict[model_version][s][k] for s in test_ids_list[k]}
    # Generate surrogate model
    # Random permutation of fold assignments of predictions
    pred_dict_permuted = {}
    for model_version in models:
        pred_dict_permuted[model_version] = {}
        for i_sub, subject_id in enumerate(dataset.all_ids):
            byfold_preds = pred_dict[model_version][subject_id]
            subject_folds = list(byfold_preds.keys())
            subject_preds = [byfold_preds[k] for k in subject_folds]
            subject_folds = np.random.RandomState(seed=i_sub).permutation(subject_folds)
            pred_dict_permuted[model_version][subject_id] = {k: pred for k, pred in zip(subject_folds, subject_preds)}
    pred_dict_permuted_original = {}
    for model_version in models:
        pred_dict_permuted_original[model_version] = {}
        for k in range(n_folds):
            pred_dict_permuted_original[model_version][k] = {s: pred_dict_permuted[model_version][s][k] for s in test_ids_list[k]}
            
    # Performance
    # AND: ensemble of stamps -> thr = 1.0
    # OR: ensemble of stamps -> thr = 1 / n_models
    # AVG: ensemble of probas with thr of 0.5
    table = {'Ensemble': [], 'F1-score': [], 'Recall': [], 'Precision': [], 'mIoU': [], 'Fold': []}
    pred_sources = {
        'v2_time': pred_dict_original['v2_time'],
        'v2_time_perm': pred_dict_permuted_original['v2_time'],
        'v2_cwt1d': pred_dict_original['v2_cwt1d'],
        'v2_cwt1d_perm': pred_dict_permuted_original['v2_cwt1d'],
    }
    pairs = [
        ('v2_time', 'v2_time'),
        ('v2_cwt1d', 'v2_cwt1d'),
        ('v2_time', 'v2_time_perm'),
        ('v2_cwt1d', 'v2_cwt1d_perm'),
        ('v2_time', 'v2_cwt1d'),
    ]
    for k in range(n_folds):
        subject_ids = test_ids_list[k]
        print("Fold %s" % (k))
        reference_feeder_dataset = FeederDataset(dataset, subject_ids, constants.N2_RECORD, which_expert=config["expert"])
        events_list = reference_feeder_dataset.get_stamps()
        for pred_source_name_1, pred_source_name_2 in pairs:
            pred_source_1 = pred_sources[pred_source_name_1]
            pred_source_2 = pred_sources[pred_source_name_2]
            if pred_source_name_1 == pred_source_name_2:
                detections_list = [pred_source_1[k][subject_id]['stamp'] for subject_id in subject_ids]
                performance = fig_utils.compute_fold_performance(events_list, detections_list, average_mode)
                ensemble_str = pred_source_name_1
                table['Ensemble'].append(ensemble_str)
                table['F1-score'].append(performance['F1-score'])
                table['Recall'].append(performance['Recall'])
                table['Precision'].append(performance['Precision'])
                table['mIoU'].append(performance['mIoU'])
                table['Fold'].append(k)
            else:
                for criterion in ensemble_criterion_list:
                    if criterion == 'and':
                        dict_of_stamps = {s: [pred_source_1[k][s]['stamp'], pred_source_2[k][s]['stamp']] for s in subject_ids}
                        ensemble_pred_obj = det_utils.generate_ensemble_from_stamps(dict_of_stamps, reference_feeder_dataset, skip_setting_threshold=True)
                        ensemble_pred_obj.set_parent_dataset(dataset)
                        ensemble_pred_obj.set_probability_threshold(1.0)
                    elif criterion == 'or':
                        dict_of_stamps = {s: [pred_source_1[k][s]['stamp'], pred_source_2[k][s]['stamp']] for s in subject_ids}
                        ensemble_pred_obj = det_utils.generate_ensemble_from_stamps(dict_of_stamps, reference_feeder_dataset, skip_setting_threshold=True)
                        ensemble_pred_obj.set_parent_dataset(dataset)
                        ensemble_pred_obj.set_probability_threshold(0.5)
                    elif criterion == 'avg':
                        dict_of_probas = {s: [pred_source_1[k][s]['probability'], pred_source_2[k][s]['probability']] for s in subject_ids}
                        ensemble_pred_obj = det_utils.generate_ensemble_from_probabilities(dict_of_probas, reference_feeder_dataset, skip_setting_threshold=True)
                        ensemble_pred_obj.set_parent_dataset(dataset)
                        ensemble_pred_obj.set_probability_threshold(0.5)
                    else:
                        raise ValueError()
                    detections_list = ensemble_pred_obj.get_stamps()
                    performance = fig_utils.compute_fold_performance(events_list, detections_list, average_mode)
                    ensemble_str = '%s(%s,%s)' % (criterion, pred_source_name_1, pred_source_name_2)
                    table['Ensemble'].append(ensemble_str)
                    table['F1-score'].append(performance['F1-score'])
                    table['Recall'].append(performance['Recall'])
                    table['Precision'].append(performance['Precision'])
                    table['mIoU'].append(performance['mIoU'])
                    table['Fold'].append(k)
    table = pd.DataFrame.from_dict(table)
    metrics_raw_list.append(table)
    mean_table = table.groupby(by=["Ensemble"]).mean().drop(columns=["Fold"]).add_suffix("_mean")
    std_table = table.groupby(by=["Ensemble"]).std(ddof=0).drop(columns=["Fold"]).add_suffix("_std")
    subgroup_stats_table = mean_table.join(std_table)
    subgroup_stats_table = subgroup_stats_table.reindex(sorted(subgroup_stats_table.columns), axis=1)
    subgroup_stats_table = subgroup_stats_table.reset_index()
    metrics_list.append(subgroup_stats_table)  
print("Done.")

In [ ]:
metrics_list[0]

In [ ]:
# Tabla Latex
ensembles_order = [
    "v2_time", 
    "v2_cwt1d",
    "and(v2_time,v2_time_perm)",
    "and(v2_cwt1d,v2_cwt1d_perm)",
    "and(v2_time,v2_cwt1d)",
    "avg(v2_time,v2_time_perm)",
    "avg(v2_cwt1d,v2_cwt1d_perm)",
    "avg(v2_time,v2_cwt1d)",
]
ensembles_print_name = {
    "v2_time": "REDv2-Time", 
    "v2_cwt1d": "REDv2-CWT",
    "and(v2_time,v2_time_perm)": "AND(REDv2-Time, REDv2-Time)",
    "and(v2_cwt1d,v2_cwt1d_perm)": "AND(REDv2-CWT, REDv2-CWT)",
    "and(v2_time,v2_cwt1d)": "AND(REDv2-Time, REDv2-CWT)",
    "avg(v2_time,v2_time_perm)": "AVG(REDv2-Time, REDv2-Time)",
    "avg(v2_cwt1d,v2_cwt1d_perm)": "AVG(REDv2-CWT, REDv2-CWT)",
    "avg(v2_time,v2_cwt1d)": "AVG(REDv2-Time, REDv2-CWT)",
}
print("\\toprule")
print("Datos & Detector & F1-score (\%) & mIoU (\%) \\\\")
for i_config, config in enumerate(eval_configs):
    print("\midrule")
    dataset_str = print_dataset_names[(config["dataset_name"], config["expert"])]
    metrics_dict = metrics_list[i_config].set_index('Ensemble').to_dict('index')
    for i_ens, ensemble in enumerate(ensembles_order):
        mets = metrics_dict[ensemble]
        metric_str = "$%1.1f\pm %1.1f$ & $%1.1f\pm %1.1f$" % (
            100 * mets['F1-score_mean'], 100 * mets['F1-score_std'],
            100 * mets['mIoU_mean'], 100 * mets['mIoU_std'])
        dataset_to_print = '%s\n' % dataset_str if (i_ens == 0) else ''
        row_str = "%s & %s & %s \\\\" % (dataset_to_print, ensembles_print_name[ensemble].ljust(30), metric_str)
        print(row_str)
        
    # Statistical tests
    #reference_ensemble = ensembles_order[0]
    #print("P-value test against %s" % reference_ensemble)
    #table = metrics_raw_list[i_config]
    #for ensemble in ensembles_order[1:]:
    #    print("%s:" % ensemble.ljust(50), end='')
    #    for metric_name in ["F1-score", "mIoU"]:
    #        model_metrics = table[table["Ensemble"] == ensemble][metric_name].values
    #        reference_metrics = table[table["Ensemble"] == reference_ensemble][metric_name].values
    #        pvalue = stats.ttest_ind(model_metrics, reference_metrics, equal_var=False)[1]
    #        print(" P(%s) %1.4f" % (metric_name, pvalue), end='')
    #    print("")
print("\\bottomrule")

# Perturbaciones en la entrada
Medir cambios en SubsetMetric vs perturbacion, donde subset metric es la metrica mean+-std del 5CV del dataset, y puede ser f1-score, recall, precision y mIoU (lo bueno es que las 4 están en el rango 0-1).
- Escalar la señal de entrada. Curva continua
- Inversión de voltaje (multiplicar por -1) y de tiempo (flipped signal y labels, para implementarlo se podria hacer flip de la entrada, y el vector de probabilidad de salida volver a hacerle un flip). Discreto (barras).
- Filtrar (quitar) bandas de potencia (modelado ya entrenado). Delta separarlo en delta lenta y rapida. Discreto (barras).

In [15]:
models = [constants.V2_TIME, constants.V2_CWT1D]
print_model_names = {
    constants.V2_TIME: 'REDv2-Time',
    constants.V2_CWT1D: 'REDv2-CWT'
}
print_dataset_names = {
    (constants.MASS_SS_NAME, 1): "MASS-SS2-E1SS",
    (constants.MASS_SS_NAME, 2): "MASS-SS2-E2SS",
    (constants.MASS_KC_NAME, 1): "MASS-SS2-KC",
    (constants.MODA_SS_NAME, 1): "MASS-MODA",
    (constants.INTA_SS_NAME, 1): "INTA-UCH",
}

eval_configs = [
    dict(dataset_name=constants.MODA_SS_NAME, expert=1, strategy='5cv', seeds=3),
    # dict(dataset_name=constants.MASS_KC_NAME, expert=1, strategy='5cv', seeds=3),
]
metrics_list = []
metrics_raw_list = []
for config in eval_configs:
    print("\nLoading", config)
    dataset = reader.load_dataset(config["dataset_name"], verbose=False)
    _, _, test_ids_list = get_partitions(dataset, config["strategy"], config["seeds"])
    n_folds = len(test_ids_list)
    average_mode = constants.MICRO_AVERAGE if (config["dataset_name"] == constants.MODA_SS_NAME) else constants.MACRO_AVERAGE
    # Collect predictions
    pred_dict = {}
    for model_version in models:
        pred_dict[model_version] = {}
        tmp_pert_dict = fig_utils.get_red_predictions_for_perturbations(model_version, config["strategy"], dataset, config["expert"], verbose=False)
        perturbations = list(tmp_pert_dict.keys())
        for perturbation_name in perturbations:
            pred_dict[model_version][perturbation_name] = {}
            tmp_dict = tmp_pert_dict[perturbation_name]
            for k in tmp_dict.keys():
                fold_subjects = tmp_dict[k][constants.TEST_SUBSET].all_ids
                fold_predictions = tmp_dict[k][constants.TEST_SUBSET].get_stamps()
                pred_dict[model_version][perturbation_name][k] = {s: pred for s, pred in zip(fold_subjects, fold_predictions)}

    # Performance
    table = {'Detector': [], 'Perturbación': [], 'F1-score': [], 'Recall': [], 'Precision': [], 'mIoU': [], 'Fold': []}
    # Measure performance of model with itself
    for model_version in models:
        for perturbation_name in perturbations:
            for k in range(n_folds):
                subject_ids = test_ids_list[k]
                feed_d = FeederDataset(dataset, subject_ids, constants.N2_RECORD, which_expert=config["expert"])
                events_list = feed_d.get_stamps()
                detections_list = [pred_dict[model_version][perturbation_name][k][subject_id] for subject_id in subject_ids]
                performance = fig_utils.compute_fold_performance(events_list, detections_list, average_mode)
                table['Detector'].append(model_version)
                table['Perturbación'].append(perturbation_name)
                table['F1-score'].append(performance['F1-score'])
                table['Recall'].append(performance['Recall'])
                table['Precision'].append(performance['Precision'])
                table['mIoU'].append(performance['mIoU'])
                table['Fold'].append(k)
    table = pd.DataFrame.from_dict(table)
    print("By-fold statistics")
    metrics_raw_list.append(table)
    mean_table = table.groupby(by=["Detector", "Perturbación"]).mean().drop(columns=["Fold"]).add_suffix("_mean")
    std_table = table.groupby(by=["Detector", "Perturbación"]).std(ddof=0).drop(columns=["Fold"]).add_suffix("_std")
    subgroup_stats_table = mean_table.join(std_table)
    subgroup_stats_table = subgroup_stats_table.reindex(sorted(subgroup_stats_table.columns), axis=1)
    subgroup_stats_table = subgroup_stats_table.reset_index()
    metrics_list.append(subgroup_stats_table)  
print("Done.")


Loading {'dataset_name': 'moda_ss', 'expert': 1, 'strategy': '5cv', 'seeds': 3}
By-fold statistics
Done.


In [16]:
metrics_list[0]

,Detector,Perturbación,F1-score_mean,F1-score_std,Precision_mean,Precision_std,Recall_mean,Recall_std,mIoU_mean,mIoU_std
0,v2_cwt1d,filter-0-2,0.801226,0.016089,0.774064,0.032434,0.831731,0.020200,0.820829,0.004457
1,v2_cwt1d,filter-10-16,0.801226,0.016089,0.774064,0.032434,0.831731,0.020200,0.820829,0.004457
2,v2_cwt1d,filter-16-30,0.801226,0.016089,0.774064,0.032434,0.831731,0.020200,0.820829,0.004457
3,v2_cwt1d,filter-2-4,0.801226,0.016089,0.774064,0.032434,0.831731,0.020200,0.820829,0.004457
4,v2_cwt1d,filter-4-8,0.801226,0.016089,0.774064,0.032434,0.831731,0.020200,0.820829,0.004457
5,v2_cwt1d,filter-8-11,0.801226,0.016089,0.774064,0.032434,0.831731,0.020200,0.820829,0.004457
6,v2_cwt1d,invert-time,0.809901,0.014368,0.803698,0.027228,0.817192,0.021707,0.776005,0.007703
7,v2_cwt1d,invert-value,0.807943,0.015894,0.812545,0.024805,0.804464,0.027824,0.822319,0.003842
8,v2_cwt1d,scale-0.5,0.786527,0.015281,0.715885,0.031003,0.874896,0.029549,0.820088,0.005820
9,v2_cwt1d,scale-0.6,0.786527,0.015281,0.715885,0.031003,0.874896,0.029549,0.820088,0.005820
